# Mask R-CNN - Nephrology Inference Tool
This is an custom version of [Mask R-CNN - Train cell nucleus Dataset](https://colab.research.google.com/github/navidyou/Mask-RCNN-implementation-for-cell-nucleus-detection-executable-on-google-colab-/blob/master/mask_RCNN_cell_nucleus_google_colab.ipynb) for Google Colab. 

If using this notebook on Google Colab, GPU/TPU might not be used due to version of TensorFlow.

## Google Colab Only

Execute only if using this notebook on Google Colab (installing compatible librairies and getting files needed). Errors might appear, do not worry about this.

In [1]:
import re
import sys
import os
IN_COLAB = 'google.colab' in sys.modules
print("Executing in Google Colab" if IN_COLAB else "Executing locally")

Executing locally


### Installing compatible librairies

In [2]:
if IN_COLAB:
    %tensorflow_version 2.x

### Retrieving needed files

You can use this cell to update the files that have been downloaded during the same session and that have been updated on GitHub.

In [3]:
if IN_COLAB:
    import os
    GITHUB_BRANCH = "experimental"
    GITHUB_REPO = f"https://raw.githubusercontent.com/AdrienJaugey/Custom-Mask-R-CNN-for-kidney-s-cell-recognition/{GITHUB_BRANCH}/"
    files = ['mrcnn/TensorflowDetector.py', 'mrcnn/utils.py', 'mrcnn/visualize.py', 'mrcnn/post_processing.py',
             'datasetTools/datasetDivider.py', 'datasetTools/datasetWrapper.py', 'datasetTools/datasetIsolator.py',
             'datasetTools/AnnotationAdapter.py', 'datasetTools/ASAPAdapter.py', 'datasetTools/LabelMeAdapter.py',
             'datasetTools/CustomDataset.py', 'nephrology.py', 'common_utils.py']
    for fileToDownload in files:
        url = GITHUB_REPO + fileToDownload
        !wget -qN $url
        if '/' in fileToDownload:
            os.makedirs(os.path.dirname(fileToDownload), exist_ok=True)
            fileName = os.path.basename(fileToDownload)
            !mv $fileName $fileToDownload

### Connecting to Google Drive

The first time this cell is executed, a link should appear, asking you to accept to give access to files of a google account. 
1.   **Follow the link**;
2.   **Choose the account** you want to link;
3.   **Accept**;
4.   **Copy the key** Google gave you;
5.   **Paste the key in the text field** that appeared below the first link you used,
6.   **Press ENTER**.

In [4]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    import shutil
    shutil.rmtree('sample_data/', ignore_errors=True)

### Retrieving your image(s)

Choose how to get your image(s) from the following list on the right   
Use ```.jp2```, ```.jpg``` or ```.png``` images only !


In [5]:
if IN_COLAB:
    howToGetImage = "From Google Drive" #@param ["Upload", "From Google Drive"]

In [6]:
if IN_COLAB:
    !rm -r images/ || true
    !mkdir -p images
    !mkdir -p images/cortex
    !mkdir -p images/main
    !mkdir -p images/mest_main
    !mkdir -p images/mest_glom

#### By upload

In [7]:
if IN_COLAB and howToGetImage == "Upload":
    imageType = 'main' #@param ["cortex", "main", "mest_main", "mest_glom"]
    print("Please upload the image(s) you want to run the inference on, you can upload the corresponding annotations files too.")
    from google.colab import files
    import shutil
    uploaded = files.upload()
    for fileName in uploaded:
        shutil.move(fileName, os.path.join("images", imageType, fileName))

#### By copy from Google Drive

Be sure to customize the 2 variables for Google Colab to be able find your file(s) in Google Drive.
Let's say you have this hierarchy in your Google Drive:
```
Root directory of Google Drive
  ├─── Directory1
  └─── Directory2
       ├─── images
       │    ├─── example1.png
       │    └─── example2.png
       └─── saved_weights
            ├─── main.h5
            └─── cortex.h5
```
1.   ```customPathInDrive``` must represent all the directories between the root directory and your image file. In the example, it would be ```Directory2/images/```. **Do not forget the final /** if you have to use this variable. Keep it empty if **file is directly in root directory** of Google Drive;
2.   ```imageFileName``` must represent the file you want to upload. In the example, it would be ```example1.png```. It can also be empty, if you want to import all the folder's images *(and annotations files if checkbox is checked)* directly to Google Colab, so in the example ```example1.png``` and ```example2.png``` would be imported.

Use the text fields available on the right.

In [8]:
if IN_COLAB and howToGetImage == "From Google Drive":
    imageType = 'main' #@param ["cortex", "main", "mest_main", "mest_glom"]
    pathToDrive = "/content/drive/MyDrive/"
    customPathInDrive = "" #@param {type:"string"}
    pathToFolder = os.path.join(pathToDrive, customPathInDrive)
    imageFilePath = "" #@param{type:"string"}
    annotationsFile = True #@param {type:"boolean"}
    if imageFilePath != "":
        pathToImage = os.path.join(pathToFolder, imageFilePath)
        tempPath = os.path.join("images", imageType, imageFilePath)
        print(f"Copying {pathToImage} to {tempPath}")
        !cp -u $pathToImage $tempPath
        if annotationsFile:
            annotationsFileName = imageFilePath.split('.')[0] + '.xml'
            pathToAnnotations = os.path.join(pathToFolder, annotationsFileName)
            tempPath = os.path.join("images", imageType, annotationsFileName)
            print(f"Copying {pathToAnnotations} to {tempPath}")
            !cp -u $pathToAnnotations $tempPath
    else:
        fileList = os.listdir(pathToFolder)
        ext = ['jp2', 'png', 'jpg']
        if annotationsFile:
            ext.extend(['xml', 'json'])
        for dataFile in fileList:
            if dataFile.split('.')[-1] in ext:
                pathToFile = "'" + os.path.join(pathToFolder, dataFile) + "'"
                tempPath = os.path.join("images", imageType, dataFile)
                print(f"Copying {pathToFile} to {tempPath}")
                !cp -u $pathToFile $tempPath

### Retrieving Weights File

Same thing than retrieving an image file using Google Drive but it is the saved weights file (```.h5``` extension) for main and cortex modes. With the past example, it would be ```Directory2/saved_weights/``` as ```customPathInDrive```, ```main.h5``` as ```mainWeightFileName``` and ```cortex.h5``` as ```cortexWeightFileName```.

In [9]:
if IN_COLAB:
    execMode = 'main' #@param ["cortex", "main", "mest_main", "mest_glom"]
    pathToDrive = "/content/drive/MyDrive/"
    # Keep customPathInDrive empty if file directly in root directory of Google Drive
    customPathInDrive = "" #@param {type:"string"}
    pathToFolder = os.path.join(pathToDrive, customPathInDrive)
    paths = {}
    paths['cortex'] = "skinet_cortex_v2" #@param {type:"string"}
    paths['main'] = "skinet_main_v1" #@param {type:"string"}
    paths['mest_main'] = "skinet_mest_main_v1" #@param {type:"string"}
    paths['mest_glom'] = "skinet_mest_glom_v1" #@param {type:"string"}
    isZipped = True  #@param {type:"boolean"}
    path = paths[execMode] + ('.zip' if isZipped else '')
    pathToWeights = os.path.join(pathToFolder, path)
    print(f"Copying {pathToWeights} to {path}")
    if isZipped:
        !cp -u $pathToWeights $path
        !unzip -q $path
    else:
        !cp -ru $pathToWeights $path

## Initialisation

### Configuration

Be sure to set ```MODEL_PATH``` to the same value than ```weightFileName```. If you want to save the results in files ```saveResults``` should be checked. You will have to open the **Files tab** in the **vertical navigation bar on the left** to see the results appearing. Then you can save them by right-clicking on each file and save it. You can customize the division size but staying at 1024 should be fine. You can also customize the thresholds and the priority table, this will change the post-processing behavior. 

```miniMaskSize``` will reduce memory consumption and duration of the processing (lower is better) however, large elements will be less smooth (reducing accuracy of the mask) (higher is better). Full resolution masks will be used in cortex mode or in main mode only if ```forceFullSizeMasks``` is set to True.

In [10]:
MODEL_PATHS = {}
MODEL_PATHS['cortex'] = "skinet_cortex_v2" #@param {type:"string"}
MODEL_PATHS['main'] = "skinet_main_v1" #@param {type:"string"}
MODEL_PATHS['mest_main'] = "skinet_mest_main_v1" #@param {type:"string"}
MODEL_PATHS['mest_glom'] = "skinet_mest_glom_v1" #@param {type:"string"}
RESULTS_PATH = "results/inference/" #@param {type:"string"}
MIN_DETECTION_CONFIDENCE = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
DIVISION_SIZE = 1024 #@param {type:"slider", min:896, max:1024, step:1}
miniMaskSize = 256 #@param {type:"slider", min:16, max:256, step:1}
forceFullSizeMasks = False  #@param {type:"boolean"}
forceLowMemoryUsage = False  #@param {type:"boolean"}
MIN_OVERLAP_PART_MAIN = 0.33 #@param {type:"slider", min:0, max:1, step:0.01}
MIN_OVERLAP_PART_CORTEX = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
displayMode = "All steps" #@param ["All steps", "Only AP & statistics"]
saveResults = True #@param {type:"boolean"}
savePreFusionImage = False #@param {type:"boolean"}
savePreFilterImage = False #@param {type:"boolean"}
minMaskArea = 50 #@param {type:"slider", min:0, max:10000000, step:50}
onBorderThreshold = 0.25  #@param {type:"slider", min:0, max:1, step:0.01}
mode = 'main' #@param ["cortex", "main", "mest_main", "mest_glom"]
if mode not in MODEL_PATHS or MODEL_PATHS[mode] == "":
    print(f"\"{mode}\" mode is not available : mode or corresponding model path is missing.")
    exit(-1)
enableCortexFusionDiv = False #@param {type:"boolean"}
MIN_DETECTION_CONFIDENCE = 0.7 if mode == "cortex" else MIN_DETECTION_CONFIDENCE
moveFusionDirToImagesAuto = False #@param {type:"boolean"}
cortexSize = (2048, 2048)
MODEL_PATH = MODEL_PATHS[mode]
classesInfo = [
    {"name": "cortex",            "color": "#ffaa00", "ignore": mode != 'cortex'},
    {"name": "medullaire",        "color": "#ff0000", "ignore": mode != 'cortex'},
    {"name": "capsule",           "color": "#ffffff", "ignore": mode != 'cortex'},

    {"name": "tubule_sain",       "color": "#ff007f", "ignore": 'main' not in mode},
    {"name": "tubule_atrophique", "color": "#55557f", "ignore": 'main' not in mode},
    {"name": "nsg",               "color": "#55007f", "ignore": 'main' not in mode},
    {"name": "nsg_complet",       "color": "#ff557f", "ignore": mode != 'main'},
    {"name": "nsg_partiel",       "color": "#55aa7f", "ignore": mode != 'main'},
    {"name": "pac",               "color": "#ffaa7f", "ignore": 'main' not in mode},
    {"name": "veine",             "color": "#0000ff", "ignore": 'main' not in mode},
    {"name": "vaisseau",          "color": "#55ff7f", "ignore": 'main' not in mode},
    {"name": "intima",            "color": "#aa0000", "ignore": mode != 'main'},
    {"name": "media",             "color": "#aa5500", "ignore": mode != 'main'},

    {"name": "hile",              "color": "#64FE2E", "ignore": mode != 'mest_glom'},
    {"name": "M",                 "color": "#55007f", "ignore": mode != 'mest_glom'},
    {"name": "E",                 "color": "#ff007f", "ignore": mode != 'mest_glom'},
    {"name": "S",                 "color": "#55557f", "ignore": mode != 'mest_glom'},
    {"name": "C",                 "color": "#ff557f", "ignore": mode != 'mest_glom'},
    {"name": "necrose_fib",       "color": "#55aa7f", "ignore": mode != 'mest_glom'}
]

infID = 1
for i in range(len(classesInfo)):
    classesInfo[i]["id"] = i
    if not classesInfo[i]["ignore"]:
        classesInfo[i]["inferenceID"] = infID
        infID += 1

# Following variables are used to fuse overlapping masks of the same class
# These thresholds determine the least part represented by the overlapping area
# of bounding boxes or masks to determine whether or not we should fuse two masks
FUSION_BB_THRESHOLD = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}
FUSION_MASK_THRESHOLD = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}

# Following variables are used to filter the masks before displaying results
# Same thing than fusion thresholds but for removing masks
FILTER_BB_THRESHOLD = 0.3 #@param {type:"slider", min:0, max:1, step:0.01}
FILTER_MASK_THRESHOLD = 0.3 #@param {type:"slider", min:0, max:1, step:0.01}

# Here are variables for full prediction mode with fusion divisions
NB_MAX_DIV_PER_AXIS = 3 #@param {type:"slider", min:3, max:7, step:1}
FUSION_DIV_THRESHOLD = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}

# Rows and columns correspond to classes in the same order than the classesInfo array without the first element
# An element set to true means that a mask of the column class, if contained by a mask of the
# row class, will be erased.
# True -> colunm element could be erased by row element
if mode == "cortex":
                      #cortex medul. capsule
    PRIORITY_TABLE = [[False, False, False], # cortex
                      [False, False, False], # medullaire
                      [False, False, False]] # capsule
elif mode == "main":
                      #tSain  tAtro  nsg    nsgC   nsgP   pac    veine  vais.  intima media
    PRIORITY_TABLE = [[False, True,  False, False, False, True,  True,  False, False, False], # tubule_sain
                      [False, False, False, False, False, True,  True,  False, False, False], # tubule_atrophique
                      [True,  True,  False, False, False, True,  True,  True,  False, False], # nsg
                      [False, False, False, False, False, False, False, False, False, False], # nsg_complet
                      [False, False, False, False, False, False, False, False, False, False], # nsg_partiel
                      [True,  True,  False, False, False, False, False, False, False, False], # pac
                      [False, False, False, False, False, False, False, False, False, False], # veine
                      [True,  True,  False, False, False, True,  True,  False, False, False], # vaisseau
                      [False, False, False, False, False, False, False, False, False, False], # intima
                      [False, False, False, False, False, False, False, False, False, False]] # media
elif mode == "mest_main":
                      #tSain  tAtro  nsg    pac    veine  vais.
    PRIORITY_TABLE = [[False, True,  False, True,  True,  False], # tubule_sain
                      [False, False, False, True,  True,  False], # tubule_atrophique
                      [True,  True,  False, True,  True,  True ], # nsg
                      [True,  True,  False, False, False, False], # pac
                      [False, False, False, False, False, False], # veine
                      [True,  True,  False, True,  True,  False]] # vaisseau
elif mode == "mest_glom":
                      #hile    M       E     S      C     necrose_fib
    PRIORITY_TABLE = [[True,  True,  True,  True,  True,  False], # hile
                      [False, False, True,  False, False, False], # M
                      [False, False, False, False, False, False], # E
                      [False, False, False, False, False, False], # S
                      [False, False, False, False, False, False], # C
                      [False, False, False, False, False, False]] # necrose_fib
else:
    raise NotImplementedError(f'{mode} mode is not implemented yet')

### Information

In [11]:
print(f"Skinet in {mode} segmentation mode :")
print("\t- Weights file : {}".format(MODEL_PATH))
if mode == 'cortex':
    print("\t- Image(s) will be resized to {} for inference.".format(cortexSize if cortexSize is not None else (1024, 1024)))
    MIN_OVERLAP_PART = MIN_OVERLAP_PART_CORTEX
else :
    print(f"\t- Masks will be {'full size' if forceFullSizeMasks else '{0}x{0} px'.format(miniMaskSize)}")
    MIN_OVERLAP_PART = MIN_OVERLAP_PART_MAIN
print("\t- Divisions will be {}x{} px with{} overlapping.".format(DIVISION_SIZE, DIVISION_SIZE, "out" if MIN_OVERLAP_PART == 0 else " {:.2%}".format(MIN_OVERLAP_PART)))

Skinet in mest_glom segmentation mode :
	- Weights file : saved_models\skinet_mest_glom_v1
	- Masks will be full size
	- Divisions will be 1024x1024 px with 33.00% overlapping.


In [12]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import nephrology as nephro
images = nephro.listAvailableImage(os.path.join('images', mode))
print("Found {} image{}{}".format(len(images) if len(images) > 0 else 'no', 's' if len(images) > 1 else '', ':' if len(images) > 0 else ''))
for image in images:
    print(" -", image)

Found 29 images:
 - images\mest_glom\ML0073B1.jpg
 - images\mest_glom\ML0073B10.jpg
 - images\mest_glom\ML0073B12.jpg
 - images\mest_glom\ML0073B2.jpg
 - images\mest_glom\ML0073B3.jpg
 - images\mest_glom\ML0073B4.jpg
 - images\mest_glom\ML0073B5.jpg
 - images\mest_glom\ML0073B6.jpg
 - images\mest_glom\ML0073B7.jpg
 - images\mest_glom\ML0073B9.jpg
 - images\mest_glom\ML0790BB9.jpg
 - images\mest_glom\ML2905M4.jpg
 - images\mest_glom\ML3771BB5.jpg
 - images\mest_glom\ML3901M1.jpg
 - images\mest_glom\ML4476B10.jpg
 - images\mest_glom\ML4476B7.jpg
 - images\mest_glom\ML5044M15.jpg
 - images\mest_glom\ML5044M2.jpg
 - images\mest_glom\ML5664BB1.jpg
 - images\mest_glom\ML5664BB3.jpg
 - images\mest_glom\ML7187M35.jpg
 - images\mest_glom\ML7379M7.jpg
 - images\mest_glom\ML8940M6.jpg
 - images\mest_glom\ML9486M7.jpg
 - images\mest_glom\MLINCO1B2.jpg
 - images\mest_glom\MLINCO6N1.jpg
 - images\mest_glom\MLINCO7N1.jpg
 - images\mest_glom\MLINCO8N1.jpg
 - images\mest_glom\MLINCO9N1.jpg


## Inference

In [13]:
import shutil
shutil.rmtree('data/', ignore_errors=True)
if len(images) > 0:
    try:
        model is None
    except NameError:
        model = nephro.NephrologyInferenceModel(classesInfo, MODEL_PATH, MIN_DETECTION_CONFIDENCE, DIVISION_SIZE, MIN_OVERLAP_PART_MAIN, MIN_OVERLAP_PART_CORTEX, cortexSize if mode == "cortex" else None, mini_mask_size=miniMaskSize, forceFullSizeMasks=forceFullSizeMasks, low_memory=IN_COLAB or forceLowMemoryUsage)
    model.inference(images, RESULTS_PATH, saveResults, FUSION_BB_THRESHOLD, FUSION_MASK_THRESHOLD, FILTER_BB_THRESHOLD, FILTER_MASK_THRESHOLD, PRIORITY_TABLE, NB_MAX_DIV_PER_AXIS, FUSION_DIV_THRESHOLD, displayOnlyAP=(displayMode == "Only AP & statistics"), allowSparse=False, savePreFusionImage=savePreFusionImage, savePreFilterImage=savePreFilterImage, minMaskArea=minMaskArea, on_border_threshold=onBorderThreshold, enableCortexFusionDiv=enableCortexFusionDiv)

Initialisation
Loading...Done !

Results will be saved to results\inference_2021-04-12_15-48-40
Using images\mest_glom\ML0073B1.jpg image file (1/29 | 03.45%)
 - Annotation file found: creating dataset files and cleaning image if possible
    - AP and confusion matrix will be computed
 - Applying annotations on file to get expected results
 - Inference ████████████████████  100.00% Duration = 08s
 - Fusing results of all divisions
 - Fusing overlapping masks ████████████████████  100.00% Duration = 00s
 - Saving pre filter image
 - Removing non-sense masks ████████████████████  100.00% Duration = 00s
 - Saving pre small masks removal image
 - Removing small masks ████████████████████  100.00% Duration = 00s
 - Saving pre fusion image
 - Computing Average Precision and Confusion Matrix
    - Average Precision is about 00.00%
 - Computing statistics on predictions
    - hile : count = 0, area = 0 px
    - M : count = 2, area = 9435 px
    - E : count = 1, area = 2321 px
    - S : count =

 - Removing small masks ████████████████████  100.00% Duration = 00s
 - Saving pre fusion image
 - Computing Average Precision and Confusion Matrix
    - Average Precision is about 00.00%
 - Computing statistics on predictions
    - hile : count = 1, area = 197737 px
    - M : count = 1, area = 4831 px
    - E : count = 0, area = 0 px
    - S : count = 0, area = 0 px
    - C : count = 0, area = 0 px
    - necrose_fib : count = 0, area = 0 px
    - nsg : count = 3, area = 1345024 px
 - Applying masks on image
 - Saving predicted annotations files
Exporting to ASAP annotation file format.
Annotations saved to results\inference_2021-04-12_15-48-40\ML0073B5\ML0073B5.xml
Exporting to LabelMe annotation file format.
Annotations saved to results\inference_2021-04-12_15-48-40\ML0073B5\ML0073B5.json
 Done in 20s

Using images\mest_glom\ML0073B6.jpg image file (8/29 | 27.59%)
 - Annotation file found: creating dataset files and cleaning image if possible
    - AP and confusion matrix will be com

 - Saving predicted annotations files
Exporting to ASAP annotation file format.
Annotations saved to results\inference_2021-04-12_15-48-40\ML3771BB5\ML3771BB5.xml
Exporting to LabelMe annotation file format.
Annotations saved to results\inference_2021-04-12_15-48-40\ML3771BB5\ML3771BB5.json
 Done in 13s

Using images\mest_glom\ML3901M1.jpg image file (14/29 | 48.28%)
 - Annotation file found: creating dataset files and cleaning image if possible
    - AP and confusion matrix will be computed
 - No mask expected, not saving expected image
 - Inference ████████████████████  100.00% Duration = 02s
 - Fusing results of all divisions
 - Fusing overlapping masks ████████████████████  100.00% Duration = 00s
 - Saving pre filter image
 - Removing non-sense masks ████████████████████  100.00% Duration = 00s
 - Saving pre small masks removal image
 - Removing small masks ████████████████████  100.00% Duration = 00s
 - Saving pre fusion image
 - Computing Average Precision and Confusion Matrix
  

 - Saving predicted annotations files
Exporting to ASAP annotation file format.
Annotations saved to results\inference_2021-04-12_15-48-40\ML5664BB1\ML5664BB1.xml
Exporting to LabelMe annotation file format.
Annotations saved to results\inference_2021-04-12_15-48-40\ML5664BB1\ML5664BB1.json
 Done in 13s

Using images\mest_glom\ML5664BB3.jpg image file (20/29 | 68.97%)
 - Annotation file found: creating dataset files and cleaning image if possible
    - AP and confusion matrix will be computed
 - No mask expected, not saving expected image
 - Inference ████████████████████  100.00% Duration = 02s
 - Fusing results of all divisions
 - Computing Average Precision and Confusion Matrix
    - Average Precision is about 100.00%
 - Computing statistics on predictions
    - hile : count = 0, area = 0 px
    - M : count = 0, area = 0 px
    - E : count = 0, area = 0 px
    - S : count = 0, area = 0 px
    - C : count = 0, area = 0 px
    - necrose_fib : count = 0, area = 0 px
    - nsg : count =

Using images\mest_glom\MLINCO6N1.jpg image file (26/29 | 89.66%)
 - Annotation file found: creating dataset files and cleaning image if possible
    - AP and confusion matrix will be computed
 - Applying annotations on file to get expected results
 - Inference ████████████████████  100.00% Duration = 02s
 - Fusing results of all divisions
 - Fusing overlapping masks ████████████████████  100.00% Duration = 00s
 - Saving pre filter image
 - Removing non-sense masks ████████████████████  100.00% Duration = 00s
 - Saving pre small masks removal image
 - Removing small masks ████████████████████  100.00% Duration = 00s
 - Saving pre fusion image
 - Computing Average Precision and Confusion Matrix
    - Average Precision is about 00.00%
 - Computing statistics on predictions
    - hile : count = 0, area = 0 px
    - M : count = 0, area = 0 px
    - E : count = 1, area = 3852 px
    - S : count = 1, area = 123381 px
    - C : count = 1, area = 147038 px
    - necrose_fib : count = 0, area = 

In [14]:
shutil.rmtree('data/', ignore_errors=True)

### Moving Fusion files (Colab + Local) and downloading results (Colab only)

In [15]:
if len(images) > 0:
    if RESULTS_PATH is None or RESULTS_PATH in ['.', './', "/", ""]:
        lastDir = "results"
        remainingPath = "./"
    else:
        if RESULTS_PATH[-1] == '/':
            index = -2
        else:
            index = -1
        lastDir = RESULTS_PATH.split('/')[index].replace('/', '')
        remainingPath = RESULTS_PATH[0:RESULTS_PATH.index(lastDir)]

    lastFoundDir = None
    fileList = os.listdir(remainingPath)
    fileList.sort()
    for resultFolder in fileList:
        if lastDir in resultFolder and os.path.isdir(os.path.join(remainingPath, resultFolder)):
            lastFoundDir = resultFolder
    if lastFoundDir is not None:
        lastFoundDirPath = os.path.join(remainingPath, lastFoundDir)
        if mode == "cortex" and moveFusionDirToImagesAuto:
                for imageFolder in os.listdir(lastFoundDirPath):
                    if os.path.isdir(os.path.join(lastFoundDirPath, imageFolder)):
                        fusionFolder = os.path.join(lastFoundDirPath, imageFolder, imageFolder + "_fusion")
                        if os.path.exists(fusionFolder):
                            if len(os.listdir(fusionFolder)) > 2:
                                skinetFile = os.path.join(lastFoundDirPath, imageFolder, imageFolder + "_fusion_info.skinet")
                                shutil.copytree(fusionFolder, os.path.join("images", "main", imageFolder + "_fusion"))
                                shutil.copy2(skinetFile, os.path.join("images", "main", imageFolder + "_fusion_info.skinet"))
        if IN_COLAB:
            zipName = lastFoundDir + '.zip'
            !zip -qr $zipName $lastFoundDirPath
            print("Results can be downloaded on the Files tab on the left")
            print("Zip file name is :", zipName)

This cell may run indefinitely, Google Colab has problem with downloading automatically large files.

In [16]:
if IN_COLAB and len(images) > 0:
    from google.colab import files
    files.download(zipName)